In [1]:
"""
Test Script

"""
import random
import copy
import numpy as np
from vwgym import VacuumWorld, Vectorise, StepWrapper
from vwgym.fun_lite import *
from torch.distributions import Categorical

import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.gymutils as gu

random.seed(123518)

if torch.cuda.is_available():
    print('GPU Available:\t', True)
    device = 'cuda'
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

else:
    device = 'cpu'

GPU Available:	 True


In [2]:
model_path = 'vwgym/saved_model/vwgym_f_net_last_checkpoint.pt'
checkpoint = torch.load(model_path)

In [3]:
args = checkpoint['args']

In [4]:
# env_n = VacuumWorld(args['grid_size'])
# env_v = Vectorise(copy.deepcopy(env_n))
# env = StepWrapper(env_v)
env, input_shape = make_env(args['grid_size'], 1)

In [5]:
num_actions = env[0].action_space.n
input_dim = (3, 8, 8)
f_net = FuNet(input_dim, args['d'], args['len_hist'], args['eps'], args['k'], num_actions, 1, device)
optimizer = torch.optim.Adam(f_net.parameters(), lr=args['lr'], eps=1e-5)

f_net.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optim'])

In [10]:
f_net

FuNet(
  (f_percept): Percept(
    (conv_1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1))
    (conv_2): Conv2d(256, 128, kernel_size=(2, 2), stride=(1, 1))
    (max_pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc_1): Linear(in_features=128, out_features=256, bias=True)
    (fc_2): Linear(in_features=256, out_features=256, bias=True)
    (percept): Linear(in_features=256, out_features=256, bias=True)
    (flat): Flatten()
  )
  (manager): Manager(
    (M_space): Linear(in_features=256, out_features=256, bias=True)
    (M_relu): ReLU()
    (M_tanh): Tanh()
    (M_goals): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
    )
    (M_value): Linear(in_features=256, out_features=1, bias=True)
  )
  (worker): Worker(
    (f_stateW): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): 

In [6]:
with torch.no_grad():
    f_net.eval()
    predictions = []
    
    goal_history, s_Mt_hist, ep_binary = f_net.agent_model_init()
    x = np.array([e.reset() for e in env])
    
    for e in env:
        e.rw_dirts = e.dirts
        print(f'Dirts Present..:{e.rw_dirts}')
        
    x = torch.from_numpy(x/255).float().to(device)
    step = 0
    prev_action = []
    
    for __ in range(5000):
        action_probs, v_Mt, v_Wt, goal_history, s_Mt_hist = f_net(x, goal_history, s_Mt_hist)
        a_t, _, _ = take_action(action_probs)
        x, reward, done, ep_info = take_step(a_t, env, device)
        for ep_d in ep_info:
            if ep_d['ep_rewards'] is not None:
                print(f"reward = {round(ep_d['ep_rewards'], 2)} \t| ep_score = {round(600/ep_d['ep_len'], 2)}")
        
        if done[0]:
            break
        ep = torch.FloatTensor(1.0 - done).unsqueeze(-1).to(device)
        ep_binary.pop(0)
        ep_binary.append(ep)
        predictions.append(a_t[0])        

Dirts Present..:16

Grid Cleaned !!

{'ep_rewards': 165, 'ep_len': 3586, 'move': 769, 'clean': 668, 'turn_left': 713, 'turn_right': 705, 'idle': 731}
Episode Rewards:	 165
reward = 165 	| ep_score = 0.17


In [7]:
import pandas as pd

In [9]:
p = pd.DataFrame.from_dict({'actions': predictions, 'action_meanings':[env[0].action_meanings[i] for i in predictions]})
p['action_meanings'].value_counts().reset_index()

,index,action_meanings
0,move,769
1,idle,731
2,turn_left,713
3,turn_right,705
4,clean,667


In [ ]:
# print(len(predictions))
# predictions.append(4)
# print(len(predictions))
# print(env_v.reset())

In [ ]:
# ac_1 = iter(predictions)
# ac_2 = iter(predictions)

# nstate = gu.episode(env_n, lambda _: next(ac_1), max_length=len(predictions))[0]
# vstate = gu.episode(env_v, lambda _: next(ac_2), max_length=len(predictions))[0]

# vis = np.concatenate([vstate[:,None,i] for i in [0,1,2]], axis=3)

# J.images(vis, scale=40, on_interact=list(zip(np.array(env.action_meanings)[predictions],nstate)));

In [ ]:
# torch.dot(torch.tensor([2, 3]), torch.tensor([2, 1]))

# a, b = torch.rand(1, 8), torch.rand(8, 5)

# nn.functional.softmax(torch.mm(a, b))

# import time

# time.sleep()